In [1]:
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
!pip install --upgrade scipy networkx numpy

2.2.1+cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 30.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 46.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.11.4
    Uninstalling scipy-1.11.4:
      Successfully uninstalled scipy-1.11.4


In [8]:

import torch
import torch.nn as nn
from torch.autograd import Variable
import math
import torch.nn.functional as F
from torch.nn.parameter import Parameter
import torch_geometric
#from torch_geometric.utils.convert import to_scipy_sparse_matrix
#from torch_geometric.utils.train_test_split_edges import torch_geometric
from scipy.sparse import csr_matrix
import networkx as nx
import numpy as np
from torch_geometric.nn import GCNConv
from importlib import reload
from torch_geometric.datasets import TUDataset
from sklearn import metrics
from time import perf_counter
import statistics
import torch_geometric.nn as gnn

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
import sys
py_path = '/content/drive/MyDrive/Explainability Methods/Models/Script/Layers/'
sys.path.insert(0,py_path)
#import matrix_util as Mat_Util
import DGCNN_layer as dgcnn_layer
import DGCNN_GNN_Layers as dgcnn_gnn_layers
import SortPooling_Layer as sortpooling_layer
import MLP_DGCNN as mlp_dgcnn



In [11]:
class MySpMM(torch.autograd.Function):
	@staticmethod
	def forward(ctx, sp_mat, dense_mat):
		ctx.save_for_backward(sp_mat, dense_mat)
		return torch.bmm(sp_mat, dense_mat)

	@staticmethod
	def backward(ctx, grad_output):
		sp_mat, dense_mat = ctx.saved_variables
		grad_matrix1 = grad_matrix2 = None

		assert not ctx.needs_input_grad[0]
		if ctx.needs_input_grad[1]:
			grad_matrix2 = Variable(torch.bmm(sp_mat.data.t(), grad_output.data), requires_grad=True)

		return grad_matrix1, grad_matrix2


def gnn_spmm(sp_mat, dense_mat):
	return MySpMM.apply(sp_mat, dense_mat)

In [86]:
import torch
import torch.nn as nn
#import matrix_util as Mat_Util
from torch.autograd import Variable
class GNN_DGCNN(nn.Module):
    '''
		A single DGCNN Layer, using propagation matrix defined by Zhang et al. in DGCNN
	'''
    def __init__(self, input_dim, latent_dim, Bias):

        super(GNN_DGCNN, self).__init__()
        self.input_dim = input_dim
        self.latent_dim = latent_dim
        self.conv_params = nn.Linear(input_dim, latent_dim, bias=Bias)


    def forward(self, input_tensor, tilda_adjacency_matrix, reciprocal_tilda_degree_matrix):

        adjacency_matrix_multiplied = torch.bmm(tilda_adjacency_matrix, input_tensor)  # Y = A~ * X
        node_linear = self.conv_params(adjacency_matrix_multiplied) # Y = Y * W
        normalized_linear = torch.bmm(reciprocal_tilda_degree_matrix, node_linear)  # Y = D^-1 * Y

        return normalized_linear

In [15]:
from torch_geometric.loader import DataLoader
mutag_dataset = TUDataset(root='data/TUDataset', name='MUTAG')
mutag_batched_dataset = DataLoader(mutag_dataset, batch_size=10, shuffle=False)
#DGCNN_layer = dgcnn_layer.GraphConvolutionLayer_DGCNN(7, 7)

In [107]:
#import DGCNN_layer as dgcnn_layer
from scipy.sparse import csr_matrix
class dgcnn_gnn_layers(nn.Module):
    def __init__(self, GNN_layers, node_feat_size, Bias, dgcnn_act_fun):
        super(dgcnn_gnn_layers, self).__init__()
        self.GNN_layers = GNN_layers
        self.num_GNN_layers = len(GNN_layers)
        self.node_feat_size = node_feat_size
        self.output_dim = GNN_layers[-1]
        self.Bias = Bias

        self.gnn_layers = []

        for i in range(self.num_GNN_layers):
            if self.num_GNN_layers == 1:
                self.gnn_layers.append(GNN_DGCNN(input_dim=self.node_feat_size, latent_dim=self.output_dim, Bias=self.Bias))
            elif self.num_GNN_layers > 1:
                if i == 0:
                    self.gnn_layers.append(GNN_DGCNN(input_dim=self.node_feat_size, latent_dim=self.GNN_layers[i], Bias=self.Bias))
                elif 0 < i < self.num_GNN_layers-1:
                    self.gnn_layers.append(GNN_DGCNN(input_dim=self.GNN_layers[i-1], latent_dim=self.GNN_layers[i], Bias=self.Bias))
                elif i == self.num_GNN_layers-1:
                    self.gnn_layers.append(GNN_DGCNN(input_dim=self.GNN_layers[i-1], latent_dim=self.output_dim, Bias=self.Bias))
            else:
                print("please enter layer config")
        self.gnn_layers = nn.Sequential(*self.gnn_layers)

        if dgcnn_act_fun == 'ReLu':
            self.dgcnn_act_fun = F.relu
            print('ReLu is Selected.')
        elif dgcnn_act_fun == 'eLu':
            self.dgcnn_act_fun = nn.functional.elu
            print('eLu is Selected.')
        elif dgcnn_act_fun == 'tanh':
            self.dgcnn_act_fun = torch.tanh
            print('tanh is Selected.')


    def computational_matrices(self, batched_graphs, edge_mask):
        if edge_mask == None:
            tilda_adjacency_matrix = torch.tensor(to_scipy_sparse_matrix(batched_graphs.edge_index).todense()) + torch.eye(len(torch.tensor(to_scipy_sparse_matrix(batched_graphs.edge_index).todense())))
        else:
            tilda_adjacency_matrix = torch.tensor(csr_matrix((np.array(edge_mask), (np.array(batched_graphs.edge_index[0]), np.array(batched_graphs.edge_index[1])))).todense()) + torch.eye(len(torch.tensor(to_scipy_sparse_matrix(batched_graphs.edge_index).todense())))
        tilda_adjacency_matrix = tilda_adjacency_matrix.type(torch.float32)

        if batched_graphs.batch is not None:
            graph_sizes = [len(batched_graphs[i].x) for i in range(len(batched_graphs))]
            batch_size = batched_graphs.num_graphs
        else:
            graph_sizes = [len(batched_graphs.x)]
            batch_size = 1
        max_number_of_nodes_in_batch_of_graphs = max(graph_sizes)

        adjacency_list = []
        degree_list = []
        feature_list = []
        start = 0
        for i in range(batch_size):
            end = start + graph_sizes[i]
            un_padded_adj = tilda_adjacency_matrix[start:end, start:end]
            adj_off_set = max_number_of_nodes_in_batch_of_graphs - un_padded_adj.size()[0]
            if un_padded_adj.size()[0] <= max_number_of_nodes_in_batch_of_graphs:
                un_padded_adj = F.pad(un_padded_adj, (0, adj_off_set, 0, adj_off_set), mode='constant', value=0)
                un_padded_adj = un_padded_adj.type(torch.float32)
                tilda_degree_vector = torch.sum(un_padded_adj, dim=1)
                num_nodes = max_number_of_nodes_in_batch_of_graphs
                tilda_degree_matrix = torch.zeros(num_nodes, num_nodes)
                tilda_degree_matrix.as_strided([num_nodes], [num_nodes + 1]).copy_(tilda_degree_vector)
            tilda_degree_matrix = tilda_degree_matrix.type(torch.float32)
            un_padded_adj = un_padded_adj.type(torch.float32)
            adjacency_list.append(un_padded_adj)
            degree_list.append(tilda_degree_matrix)
            un_padded_feat = batched_graphs.x[start:end, :]
            node_feat_off_set = max_number_of_nodes_in_batch_of_graphs - graph_sizes[i]
            un_padded_feat = F.pad(un_padded_feat, (0, 0, 0, node_feat_off_set), mode='constant', value=0)
            un_padded_feat = un_padded_feat.type(torch.float32)
            un_padded_feat.require_grad = True
            feature_list.append(un_padded_feat)
            start = end

        adjacency_list = list(map(lambda x: torch.unsqueeze(x, 0), adjacency_list))
        degree_list = list(map(lambda x: torch.unsqueeze(x, 0), degree_list))
        feature_list = list(map(lambda x: torch.unsqueeze(x, 0), feature_list))

        new_adjacecny = torch.cat(adjacency_list, dim=0)
        new_degree = torch.cat(degree_list, dim=0)
        new_features = torch.cat(feature_list, dim=0)

        reciprocal_tilda_degree_matrix = torch.reciprocal(new_degree)
        reciprocal_tilda_degree_matrix = torch.nan_to_num(reciprocal_tilda_degree_matrix, nan=0, neginf=0.0, posinf=0.0)

        return new_adjacecny, reciprocal_tilda_degree_matrix, new_features

    def forward(self, graph, edge_mask):
        x, edge_index, batch, y = graph.x, graph.edge_index, graph.batch, graph.y

        if batch is not None:
            graph_sizes = [len(graph[i].x) for i in range(len(graph))]
        else:
            graph_sizes = [len(graph.x)]

        Output_of_GNN_Layers = []
        new_adjacecny, reciprocal_tilda_degree_matrix, x = self.computational_matrices(graph, edge_mask)

        for i in range(self.num_GNN_layers):
            x = self.gnn_layers[i](x, new_adjacecny, reciprocal_tilda_degree_matrix)
            x = self.dgcnn_act_fun(x)
            Output_of_GNN_Layers.append(x)
        return x#Output_of_GNN_Layers



dgcnn_layers_example = dgcnn_gnn_layers(GNN_layers=[32, 32, 32, 7], node_feat_size=7, Bias=True, dgcnn_act_fun="tanh")


tanh is Selected.


In [82]:
example = mutag_dataset[0]
e_ind = [[0, 0, 1], [1, 2, 0]]
e_ind = np.array(e_ind)
e_ind_t = torch.from_numpy(e_ind)
example.edge_index = e_ind_t
print(example.edge_index)
print(example.x.size())
print(example)

tensor([[0, 0, 1],
        [1, 2, 0]])
torch.Size([17, 7])
Data(edge_index=[2, 3], x=[17, 7], edge_attr=[38, 4], y=[1])


In [109]:
Output_of_GNN_Layers = dgcnn_layers_example(example, None)
print(Output_of_GNN_Layers)

tensor([[[-0.0160,  0.0086,  0.1299,  0.0369,  0.0136, -0.0698,  0.0021],
         [-0.0520,  0.0189,  0.1408,  0.0292,  0.0249, -0.0898,  0.0010],
         [-0.0868,  0.0278,  0.2448,  0.0391,  0.0078, -0.1711, -0.0630],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  

In [99]:
from torch_geometric.loader import DataLoader
dataset = TUDataset(root='data/TUDataset', name='MUTAG')
batched_dataset = DataLoader(dataset, batch_size=10, shuffle=False)

In [110]:
for i,batch in enumerate(mutag_batched_dataset):
    #print(batch)
    print(batch.x.size())
    Output_of_GNN_Layers = dgcnn_layers_example(batch, None)
    print("here:", len(Output_of_GNN_Layers), Output_of_GNN_Layers.size())
    print(Output_of_GNN_Layers)
    break

    #if i == 176:
    #    #print(i, Output_of_GNN_Layers[-1],tilda_degree_matrix, reciprocal_tilda_degree_matrix)
    #    print(i, Output_of_GNN_Layers)

torch.Size([166, 7])
here: 10 torch.Size([10, 28, 7])
tensor([[[-0.0414,  0.0006,  0.1046,  ...,  0.0368, -0.0622,  0.0056],
         [-0.0414,  0.0006,  0.1045,  ...,  0.0369, -0.0621,  0.0055],
         [-0.0430,  0.0014,  0.1028,  ...,  0.0378, -0.0616,  0.0072],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

        [[-0.0415,  0.0029,  0.1026,  ...,  0.0350, -0.0625,  0.0024],
         [-0.0420,  0.0047,  0.1019,  ...,  0.0358, -0.0626,  0.0056],
         [-0.0306,  0.0046,  0.0881,  ...,  0.0327, -0.0510,  0.0097],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

        [[-0.0411,  0.0045,  0.1021,  ...,  0.0340, -0.0629,  

In [111]:

import os
import sys
import numpy as np
import torch
import torch.nn as nn

class SortPooling_updated(nn.Module):
    def __init__(self, sort_pooling_k, node_feat_size):
        super(SortPooling_updated, self).__init__()
        self.sort_pooling_k, = sort_pooling_k,
        self.node_feat_size = node_feat_size


    def max_sort(self, row):
        return torch.max(row)

    #def index_select(self, tensor, dim, index):
    #    return tensor.gather(dim, index.unsqueeze(dim)).squeeze(dim)

    def forward(self, output_of_dgcnn_layer, batch_graphs):
        if batch_graphs.batch is not None:
            graph_sizes = [len(batch_graphs[i].x) for i in range(len(batch_graphs))]
        else:
            graph_sizes = [len(batch_graphs.x)]


        start_index = 0
        batch_graphs_ready_to_sort = []
        if batch_graphs.batch is not None:
            if output_of_dgcnn_layer.size()[1] >= self.sort_pooling_k:

                values, indexes = torch.topk(output_of_dgcnn_layer, k=1, dim=-1)
                vals, indxs = torch.topk(values, k=self.sort_pooling_k, dim=1)
                indxs = torch.squeeze(indxs, -1)
                dummy = indxs.unsqueeze(2).expand(indxs.size(0), indxs.size(1), output_of_dgcnn_layer.size(2))
                sorted_graph = torch.gather(output_of_dgcnn_layer, 1, dummy)

            else:
                off_set = self.sort_pooling_k - output_of_dgcnn_layer.size()[1]
                output_of_dgcnn_layer = F.pad(output_of_dgcnn_layer, (0, 0, 0, off_set), mode='constant', value=0)
                output_of_dgcnn_layer = output_of_dgcnn_layer.type(torch.float32)


                values, indexes = torch.topk(output_of_dgcnn_layer, k=1, dim=-1)
                vals, indxs = torch.topk(values, k=output_of_dgcnn_layer.size()[1], dim=1)
                indxs = torch.squeeze(indxs, -1)
                dummy = indxs.unsqueeze(2).expand(indxs.size(0), indxs.size(1), output_of_dgcnn_layer.size(2))
                sorted_graph = torch.gather(output_of_dgcnn_layer, 1, dummy)

                batch_graphs_ready_to_sort.append(sorted_graph)


        else:
            for i in range(len([batch_graphs])):

                if output_of_dgcnn_layer[i].size()[0] >= self.sort_pooling_k:
                    top_max_values, top_max_indexes = torch.topk(output_of_dgcnn_layer[i], k=1, dim=1)
                    top_k_values, top_k_indexes = torch.topk(top_max_values, k=self.sort_pooling_k, dim=0)

                    sorted_graph = output_of_dgcnn_layer[i].index_select(dim=0, index=torch.reshape(top_k_indexes, (-1,)))

                else:
                    off_set = self.sort_pooling_k - output_of_dgcnn_layer[i].size()[0]
                    output_of_dgcnn_layer = F.pad(output_of_dgcnn_layer, (0, 0, 0, off_set), mode='constant', value=0)
                    output_of_dgcnn_layer = output_of_dgcnn_layer.type(torch.float32)

                    top_max_values, top_max_indexes = torch.topk(output_of_dgcnn_layer[i], k=1, dim=1)
                    top_k_values, top_k_indexes = torch.topk(top_max_values, k=output_of_dgcnn_layer[i].size()[0], dim=0)
                    sorted_graph = output_of_dgcnn_layer[i].index_select(dim=0, index=torch.reshape(top_k_indexes, (-1,)))


        return sorted_graph


In [112]:
sortpooling_exaple = SortPooling_updated(sort_pooling_k=17, node_feat_size=7)

In [113]:
for i,batch in enumerate(dataset):
    #print(batch)

    Output_of_GNN_Layers = dgcnn_layers_example(batch, None)

    output_of_sortpooling = sortpooling_exaple(Output_of_GNN_Layers, batch)


    #print(Output_of_GNN_Layers[-1])

    #if i == 176:
    #    #print(i, Output_of_GNN_Layers[-1],tilda_degree_matrix, reciprocal_tilda_degree_matrix)
    #    print(i, Output_of_GNN_Layers)

In [114]:
example = torch.arange(70)
example = torch.reshape(example, (10, 7))
print(example)

example2 = F.pad(example, (0, 1, 0, 1), mode='constant', value=0)
example2 = example2.type(torch.float32)
example2 = Variable(example2, requires_grad=True)
print(example2)




tensor([[ 0,  1,  2,  3,  4,  5,  6],
        [ 7,  8,  9, 10, 11, 12, 13],
        [14, 15, 16, 17, 18, 19, 20],
        [21, 22, 23, 24, 25, 26, 27],
        [28, 29, 30, 31, 32, 33, 34],
        [35, 36, 37, 38, 39, 40, 41],
        [42, 43, 44, 45, 46, 47, 48],
        [49, 50, 51, 52, 53, 54, 55],
        [56, 57, 58, 59, 60, 61, 62],
        [63, 64, 65, 66, 67, 68, 69]])
tensor([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  0.],
        [ 7.,  8.,  9., 10., 11., 12., 13.,  0.],
        [14., 15., 16., 17., 18., 19., 20.,  0.],
        [21., 22., 23., 24., 25., 26., 27.,  0.],
        [28., 29., 30., 31., 32., 33., 34.,  0.],
        [35., 36., 37., 38., 39., 40., 41.,  0.],
        [42., 43., 44., 45., 46., 47., 48.,  0.],
        [49., 50., 51., 52., 53., 54., 55.,  0.],
        [56., 57., 58., 59., 60., 61., 62.,  0.],
        [63., 64., 65., 66., 67., 68., 69.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]], requires_grad=True)


In [ ]:
example = torch.arange(140)
example = torch.reshape(example, (2, 10, 7))
print(example)

example = F.pad(example, (0, 0, 0, 1), mode='constant', value=0)
example = example.type(torch.float32)
example = Variable(example, requires_grad=True)
print(example)

tensor([[[  0,   1,   2,   3,   4,   5,   6],
         [  7,   8,   9,  10,  11,  12,  13],
         [ 14,  15,  16,  17,  18,  19,  20],
         [ 21,  22,  23,  24,  25,  26,  27],
         [ 28,  29,  30,  31,  32,  33,  34],
         [ 35,  36,  37,  38,  39,  40,  41],
         [ 42,  43,  44,  45,  46,  47,  48],
         [ 49,  50,  51,  52,  53,  54,  55],
         [ 56,  57,  58,  59,  60,  61,  62],
         [ 63,  64,  65,  66,  67,  68,  69]],

        [[ 70,  71,  72,  73,  74,  75,  76],
         [ 77,  78,  79,  80,  81,  82,  83],
         [ 84,  85,  86,  87,  88,  89,  90],
         [ 91,  92,  93,  94,  95,  96,  97],
         [ 98,  99, 100, 101, 102, 103, 104],
         [105, 106, 107, 108, 109, 110, 111],
         [112, 113, 114, 115, 116, 117, 118],
         [119, 120, 121, 122, 123, 124, 125],
         [126, 127, 128, 129, 130, 131, 132],
         [133, 134, 135, 136, 137, 138, 139]]])
tensor([[[  0.,   1.,   2.,   3.,   4.,   5.,   6.],
         [  7.,   8.,  

In [ ]:
example = torch.arange(140)
example = torch.reshape(example, (2, 10, 7))
print(example)

pad = torch.zeros(28)
pad = torch.reshape(pad, (2, 2, 7))
print(pad)



sorted_graph = torch.cat((example, pad), 1)
print(sorted_graph)

tensor([[[  0,   1,   2,   3,   4,   5,   6],
         [  7,   8,   9,  10,  11,  12,  13],
         [ 14,  15,  16,  17,  18,  19,  20],
         [ 21,  22,  23,  24,  25,  26,  27],
         [ 28,  29,  30,  31,  32,  33,  34],
         [ 35,  36,  37,  38,  39,  40,  41],
         [ 42,  43,  44,  45,  46,  47,  48],
         [ 49,  50,  51,  52,  53,  54,  55],
         [ 56,  57,  58,  59,  60,  61,  62],
         [ 63,  64,  65,  66,  67,  68,  69]],

        [[ 70,  71,  72,  73,  74,  75,  76],
         [ 77,  78,  79,  80,  81,  82,  83],
         [ 84,  85,  86,  87,  88,  89,  90],
         [ 91,  92,  93,  94,  95,  96,  97],
         [ 98,  99, 100, 101, 102, 103, 104],
         [105, 106, 107, 108, 109, 110, 111],
         [112, 113, 114, 115, 116, 117, 118],
         [119, 120, 121, 122, 123, 124, 125],
         [126, 127, 128, 129, 130, 131, 132],
         [133, 134, 135, 136, 137, 138, 139]]])
tensor([[[0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.]

In [ ]:
example = torch.arange(70)
#print("example: ", example)
example = torch.reshape(example, (10,7))
print("example: ", example)
print()
values, indexes = torch.topk(example, k=1, dim=1)
print(values)
print()
vals, indxs = torch.topk(values, k=5, dim=0)
print(vals, indxs)

xx = example.index_select(dim=0, index=torch.reshape(indxs, (-1,)))
print(xx)

example:  tensor([[ 0,  1,  2,  3,  4,  5,  6],
        [ 7,  8,  9, 10, 11, 12, 13],
        [14, 15, 16, 17, 18, 19, 20],
        [21, 22, 23, 24, 25, 26, 27],
        [28, 29, 30, 31, 32, 33, 34],
        [35, 36, 37, 38, 39, 40, 41],
        [42, 43, 44, 45, 46, 47, 48],
        [49, 50, 51, 52, 53, 54, 55],
        [56, 57, 58, 59, 60, 61, 62],
        [63, 64, 65, 66, 67, 68, 69]])

tensor([[ 6],
        [13],
        [20],
        [27],
        [34],
        [41],
        [48],
        [55],
        [62],
        [69]])

tensor([[69],
        [62],
        [55],
        [48],
        [41]]) tensor([[9],
        [8],
        [7],
        [6],
        [5]])
tensor([[63, 64, 65, 66, 67, 68, 69],
        [56, 57, 58, 59, 60, 61, 62],
        [49, 50, 51, 52, 53, 54, 55],
        [42, 43, 44, 45, 46, 47, 48],
        [35, 36, 37, 38, 39, 40, 41]])


In [ ]:
example = torch.arange(140)
#print("example: ", example)
example = torch.reshape(example, (2, 10, 7))
print("example: ", example)
print()
values, indexes = torch.topk(example, k=1, dim=-1)
print(values)
print()
vals, indxs = torch.topk(values, k=2, dim=1)
print(vals)
print("indxs: ",indxs)
indxs = torch.squeeze(indxs, -1)
print("indxs: ",indxs)
dummy = indxs.unsqueeze(2).expand(indxs.size(0), indxs.size(1), example.size(2))
out = torch.gather(example, 1, dummy)
print(out)

example:  tensor([[[  0,   1,   2,   3,   4,   5,   6],
         [  7,   8,   9,  10,  11,  12,  13],
         [ 14,  15,  16,  17,  18,  19,  20],
         [ 21,  22,  23,  24,  25,  26,  27],
         [ 28,  29,  30,  31,  32,  33,  34],
         [ 35,  36,  37,  38,  39,  40,  41],
         [ 42,  43,  44,  45,  46,  47,  48],
         [ 49,  50,  51,  52,  53,  54,  55],
         [ 56,  57,  58,  59,  60,  61,  62],
         [ 63,  64,  65,  66,  67,  68,  69]],

        [[ 70,  71,  72,  73,  74,  75,  76],
         [ 77,  78,  79,  80,  81,  82,  83],
         [ 84,  85,  86,  87,  88,  89,  90],
         [ 91,  92,  93,  94,  95,  96,  97],
         [ 98,  99, 100, 101, 102, 103, 104],
         [105, 106, 107, 108, 109, 110, 111],
         [112, 113, 114, 115, 116, 117, 118],
         [119, 120, 121, 122, 123, 124, 125],
         [126, 127, 128, 129, 130, 131, 132],
         [133, 134, 135, 136, 137, 138, 139]]])

tensor([[[  6],
         [ 13],
         [ 20],
         [ 27],
 

In [ ]:
values, indexes = torch.topk(dataset[0].x, k=5, dim=0)
print(len(dataset[0].x), dataset[0].x,"\n","\n", values,"\n", indexes)
#print(torch.reshape(dataset[0].x, (-1,)))
xx = dataset[0].x.index_select(dim=0, index=torch.reshape(indexes, (-1,)))
print(xx.size())

17 tensor([[1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0.]]) 
 
 tensor([[1., 1., 1., 0., 0., 0., 0.],
        [1., 0., 1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.]]) 
 tensor([[11, 14, 16, 10, 10, 10, 10],
        [ 8, 10, 15, 12, 12, 12, 12],
        [ 9, 11, 11, 11, 11, 11, 11],
        [10, 13,  9, 13, 13, 13, 13],
 

In [ ]:
x = torch.tensor([[1, 2, 3, 4],[5, 6, 7 , 8],[9, 10, 11, 12],[13, 14, 15, 16]])

vals, idx = torch.topk(x, k=2, dim=0)
print(idx )
x.scatter_(0, idx, 0)
print(x)

tensor([[3, 3, 3, 3],
        [2, 2, 2, 2]])
tensor([[1, 2, 3, 4],
        [5, 6, 7, 8],
        [0, 0, 0, 0],
        [0, 0, 0, 0]])


In [ ]:
x = torch.tensor([1.0, 2], requires_grad=True)
x = x+5 # x now has a grad_fn
y = x.type(torch.double)
z = x.type(torch.uint8)

print(y)
print(z)

tensor([6., 7.], dtype=torch.float64, grad_fn=<ToCopyBackward0>)
tensor([6, 7], dtype=torch.uint8)


In [ ]:

import os
import sys
import numpy as np
import torch
import torch.nn as nn

class SortPooling_old(nn.Module):
    def __init__(self, sort_pooling_k, node_feat_size):
        super(SortPooling_old, self).__init__()
        self.sort_pooling_k, = sort_pooling_k,
        self.node_feat_size = node_feat_size


    def max_sort(self, row):
        return max(row)


    def forward(self, output_of_dgcnn_layer, batch_graphs):
        graph_sizes = [len(batch_graphs[i].x) for i in range(len(batch_graphs))]
        batch_sortpooling_zero_graphs = torch.zeros(len(graph_sizes), self.sort_pooling_k, self.node_feat_size)
        #print("batch of graphs reshaped: ", batch_sortpooling_zero_graphs.size())

        start_index = 0
        batch_graphs_ready_to_sort = []
        if batch_graphs.batch is not None:
            for i in range(len(batch_graphs)):
                graph_to_sort = output_of_dgcnn_layer[start_index: start_index + graph_sizes[i]]
                graph_to_sort = torch.reshape(graph_to_sort, (graph_sizes[i], self.node_feat_size))

                sorted_graph = torch.Tensor(sorted(graph_to_sort.cpu().detach().tolist(), key = lambda row : max(row), reverse=True))
                if sorted_graph.size()[0] >= self.sort_pooling_k:
                    batch_sortpooling_zero_graphs[i][:self.sort_pooling_k, 0:self.node_feat_size] = Variable(sorted_graph[:self.sort_pooling_k, 0:self.node_feat_size])
                    #print(batch_sortpooling_zero_graphs[i])
                else:
                    batch_sortpooling_zero_graphs[i][:sorted_graph.size()[0], 0:self.node_feat_size] = Variable(sorted_graph[:sorted_graph.size()[0], 0:self.node_feat_size])
                    #print(batch_sortpooling_zero_graphs[i])

                batch_graphs_ready_to_sort.append(sorted_graph)
                strat_index = start_index + graph_sizes[i]
        else:
            for i in range(len([batch_graphs])):
                graph_to_sort = output_of_dgcnn_layer[start_index: start_index + graph_sizes[i]]
                graph_to_sort = torch.reshape(graph_to_sort, (graph_sizes[i], self.node_feat_size))

                sorted_graph = torch.Tensor(sorted(graph_to_sort.cpu().detach().tolist(), key = lambda row : max(row), reverse=True))
                if sorted_graph.size()[0] >= self.sort_pooling_k:
                    batch_sortpooling_zero_graphs[i][:self.sort_pooling_k, 0:self.node_feat_size] = sorted_graph[:self.sort_pooling_k, 0:self.node_feat_size]
                    #print(batch_sortpooling_zero_graphs[i])
                else:
                    batch_sortpooling_zero_graphs[i][:sorted_graph.size()[0], 0:self.node_feat_size] = sorted_graph[:sorted_graph.size()[0], 0:self.node_feat_size]


                batch_graphs_ready_to_sort.append(sorted_graph)
                strat_index = start_index + graph_sizes[i]

        return batch_sortpooling_zero_graphs


In [115]:

from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F



class MLP_DGCNN(nn.Module):
    def __init__(self, num_class, node_feat_size, mlp_act_fun, dropout_rate, hid_channels, conv1d_kernels, dgcnn_k, ffn_layer_size,
                 Bias, strides):

        super(MLP_DGCNN, self).__init__()
        self.dgcnn_k = dgcnn_k
        self.node_feat_size = node_feat_size
        self.hid_channels = hid_channels
        self.conv1d_kernels = conv1d_kernels
        self.padding = 0
        self.ffn_layer_size = ffn_layer_size
        self.num_class = num_class
        self.Bias = Bias
        self.strides = strides

        self.conv1d_1 = nn.Conv1d(in_channels=1, out_channels=self.hid_channels[0], kernel_size=self.conv1d_kernels[0], stride=self.strides[0], padding=self.padding, bias=self.Bias)

        self.maxpool1d = nn.MaxPool1d(kernel_size=2, stride=2)

        self.conv1d_2 = nn.Conv1d(in_channels=self.hid_channels[0], out_channels=self.hid_channels[1], kernel_size=self.conv1d_kernels[1], stride=self.strides[1], bias=self.Bias)

        dim_conv1_out = int((self.dgcnn_k*self.node_feat_size + 2*self.padding -2)/(self.strides[0]) + 1)
        dim_conv1_out = dim_conv1_out/2

        dim_conv2_out = int((dim_conv1_out - self.conv1d_kernels[1])/(self.strides[1])+1)

        self.linear1 = nn.Linear(self.hid_channels[1]*dim_conv2_out, self.ffn_layer_size, bias=self.Bias)
        self.linear2 = nn.Linear(self.ffn_layer_size, self.num_class, bias=self.Bias)

        if mlp_act_fun == 'ReLu':
            self.mlp_act_fun = F.relu
        self.soft_fun = F.softmax
        self.dropout_linear1 = nn.Dropout(p=dropout_rate)


    def forward(self, sortpooled_embedings, graph_sizes):

        to_conv1d_1 = sortpooled_embedings.view((-1, 1, self.dgcnn_k * self.node_feat_size))

        conv1d_1_res = self.conv1d_1(to_conv1d_1)
        output_conv1d_1 = self.mlp_act_fun(conv1d_1_res)

        maxpooled_output_conv1d_1 = self.maxpool1d(output_conv1d_1)

        conv1d_2_res = self.conv1d_2(maxpooled_output_conv1d_1)
        output_conv1d_2 = self.mlp_act_fun(conv1d_2_res)

        all_but_last_two_dims = output_conv1d_2.size()[:-2]

        to_dense = output_conv1d_2.view(*all_but_last_two_dims, 1, -1)
        to_dense = torch.squeeze(to_dense, 1)

        ffn_1 = self.linear1(to_dense)
        ffn_1 = self.mlp_act_fun(ffn_1)

        dropout_ffn_1 = self.dropout_linear1(ffn_1)

        ffn_2 = self.linear2(dropout_ffn_1)
        softmaxed_ffn_2 = self.soft_fun(ffn_2, dim=1)

        return output_conv1d_1, maxpooled_output_conv1d_1, output_conv1d_2, to_dense, ffn_1, dropout_ffn_1, ffn_2, softmaxed_ffn_2


In [116]:
layers = MLP_DGCNN(node_feat_size=7, num_class=2, mlp_act_fun='ReLu', dropout_rate=0.5, Bias=False, dgcnn_k=17,
                   hid_channels=[16,32], conv1d_kernels=[2,5], ffn_layer_size=128, strides=[2,1])

In [117]:
for i,batch in enumerate(dataset):
    #print(batch)

    Output_of_GNN_Layers = dgcnn_layers_example(batch, None)
    #print("here:", len(Output_of_GNN_Layers), Output_of_GNN_Layers[0].size())
    output_of_sortpooling = sortpooling_exaple(Output_of_GNN_Layers, batch)
    #print("output_of_sortpooling: ", output_of_sortpooling.size())
    #print(Output_of_GNN_Layers.requires_grad)
    if batch.batch is not None:
        graph_sizes = [len(batch[i].x) for i in range(len(batch))]
    else:
        graph_sizes = [len(batch.x)]
    output_conv1d_1, maxpooled_output_conv1d_1, output_conv1d_2, to_dense, ffn_1, dropout_ffn_1, ffn_2, softmaxed_ffn_2 = layers(output_of_sortpooling, graph_sizes)

In [ ]:

#from torch_geometric.typing import torch_sparse
graph = dataset[0]
#adjacency_matrix = torch.tensor(to_scipy_sparse_matrix(graph.edge_index).todense())
#identity_sized_adjacency_matrix = torch.eye(len(adjacency_matrix))

#tilda_adjacency_matrix = adjacency_matrix + identity_sized_adjacency_matrix
tilda_adjacency_matrix = torch.tensor(to_scipy_sparse_matrix(graph.edge_index).todense()) + torch.eye(len(torch.tensor(to_scipy_sparse_matrix(graph.edge_index).todense())))
tilda_adjacency_matrix = tilda_adjacency_matrix.type(torch.float32)


tilda_degree_vector = torch.sum(tilda_adjacency_matrix, dim=1)

k = tilda_degree_vector.size(0)

tilda_degree_matrix = torch.zeros(k, k)
tilda_degree_matrix.as_strided([k], [k + 1]).copy_(tilda_degree_vector)
tilda_degree_matrix = tilda_degree_matrix.to(torch.float32)

input_tensor = graph.x.to(torch.float32)

ones_matrix = torch.ones_like(tilda_degree_matrix)

reciprocal_tilda_degree_matrix = ones_matrix.div(tilda_degree_matrix)
reciprocal_tilda_degree_matrix = torch.nan_to_num(reciprocal_tilda_degree_matrix, nan=0, posinf=0, neginf=0)

#node_degs = torch.transpose(torch.sum(tilda_adjacency_matrix, dim=0, keepdim=True).add(1), 0, 1)
#print(node_degs)
layer = GraphConvolutionLayer_DGCNN(7,7)
output = layer(input_tensor, tilda_adjacency_matrix, reciprocal_tilda_degree_matrix)
print("final output:", output)

NameError: ignored

In [ ]:
a = [[1, 2, 3], [4, 5], [6, 7, 8, 9,5]]
import numpy as np
b = np.zeros([len(a),len(max(a,key = lambda x: len(x)))])
print(b)
for i,j in enumerate(a):
    b[i][0:len(j)] = j


t = torch.tensor([[2., 2., 2., 2.],
                  [2., 2., 2., 2.],
                  [2., 2., 2., 2.]])
sm = F.softmax

t[0:1,0:4] = 1
t[2:3,0:4] = 3
print(t)
print(sm(t, dim=1))



[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
tensor([[1., 1., 1., 1.],
        [2., 2., 2., 2.],
        [3., 3., 3., 3.]])
tensor([[0.2500, 0.2500, 0.2500, 0.2500],
        [0.2500, 0.2500, 0.2500, 0.2500],
        [0.2500, 0.2500, 0.2500, 0.2500]])


In [ ]:
#from IPython.core.display import deepcopy
#from __future__ import print_function




#sortpooling_example = SortPooling(32, 7)
#test_list = [[5, 7, 8], [9, 10, 3],
#             [10, 18, 3], [0, 3, 5]]
#out = sortedpool([dataset[0].x])
#print(out)

In [ ]:
#[32, 32, 32, 7]
for batch_graphs in batched_dataset:
    graph_sizes = [len(batch_graphs[i].x) for i in range(len(batch_graphs))]
    print(graph_sizes)
    softed = sortpooling_example(batch_graphs, graph_sizes)
    print("softmax: ", softmaxed_h2)
    break

[17, 13, 13, 19, 11, 28, 16, 20, 12, 17, 17, 20, 22, 13, 19, 22, 11, 17, 13, 18]


NameError: ignored

In [ ]:
#sortedpool = SortPooling(2)

In [ ]:
#test_list = [[5, 7, 8], [9, 10, 3],
#             [10, 18, 3], [0, 3, 5]]
#out = sortedpool([dataset[0].x])
#print(out)

In [119]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import math
import torch.nn.functional as F
from torch.nn.parameter import Parameter
from torch_geometric.utils.convert import to_scipy_sparse_matrix
#from torch_geometric.utils.train_test_split_edges import torch_geometric
import networkx as nx
import numpy as np
from torch_geometric.nn import GCNConv
import sys
from torch_geometric.datasets import TUDataset
py_path = '/content/drive/MyDrive/Explainability Methods/Models/Script/Layers/'
sys.path.insert(0,py_path)
#import matrix_util as Mat_Util
#import DGCNN_layer as dgcnn_layer
#import DGCNN_GNN_Layers as dgcnn_gnn_layers
#import SortPooling_Layer as sortpooling_layer
#import MLP_DGCNN as mlp_dgcnn

class GlobalMeanPool(nn.Module):

    def __init__(self):
        super().__init__()

    def forward(self, x, batch):
        return gnn.global_mean_pool(x, batch)
################################################################################
class IdenticalPool(nn.Module):

    def __init__(self):
        super().__init__()

    def forward(self, x, batch):
        return x

################################################################################
class DGCNN_Model(nn.Module):
    '''
        DGCNN Layers using sparse adjacency matrix
    '''
    def __init__(self, GNN_layers, mlp_act_fun, dgcnn_act_fun, mlp_dropout_rate, Weight_Initializer, Bias, num_classes, dgcnn_k,
                 node_feat_size, hid_channels, conv1d_kernels, ffn_layer_size, strides):

        super(DGCNN_Model, self).__init__()
        self.GNN_layers = GNN_layers
        self.output_dim = GNN_layers[-1]
        self.num_GNN_layers = len(GNN_layers)
        self.mlp_dropout_rate = mlp_dropout_rate
        self.Bias = Bias
        self.Weight_Initializer = Weight_Initializer
        self.dgcnn_k = dgcnn_k
        self.node_feat_size = node_feat_size
        self.num_classes = num_classes
        self.hid_channels = hid_channels
        self.conv1d_kernels = conv1d_kernels
        self.ffn_layer_size = ffn_layer_size
        self.strides = strides

        self.gnn_layers = dgcnn_gnn_layers(GNN_layers=[32, 32, 32, 7], node_feat_size=self.node_feat_size,
                                                            Bias=self.Bias, dgcnn_act_fun=dgcnn_act_fun)

        self.sort_pool = SortPooling_updated(self.dgcnn_k, node_feat_size=self.node_feat_size)

        self.classic_conv = MLP_DGCNN(num_class=self.num_classes, node_feat_size=self.node_feat_size,
                                                mlp_act_fun=mlp_act_fun, dropout_rate=self.mlp_dropout_rate,
                                                hid_channels=self.hid_channels, conv1d_kernels=self.conv1d_kernels,
                                                dgcnn_k=self.dgcnn_k, ffn_layer_size=self.ffn_layer_size, Bias=self.Bias,
                                                strides=self.strides)
        if dgcnn_act_fun == 'ReLu':
            self.dgcnn_act_fun = F.relu
            print('ReLu is Selected.')
        elif dgcnn_act_fun == 'eLu':
            self.dgcnn_act_fun = nn.functional.elu
            print('eLu is Selected.')
        elif dgcnn_act_fun == 'tanh':
            self.dgcnn_act_fun = torch.tanh
            print('tanh is Selected.')



        mean = 0
        std = 0.1
        #self.initialize_weights(Weight_Initializer, Bias, mean, std)


    def initialize_weights(model, Weight_Initializer, Bias, mean, std):
        # 1. Xavier Normal_.  2. Kaiming Normal_.  3. Uniform (0,0.1std)
        if Weight_Initializer == 1:                                             #.      1. Xavier Normal_.
            for i,layers in enumerate(model.children()):
                if isinstance(layers, dgcnn_gnn_layers.dgcnn_gnn_layers):
                    for j, layer in enumerate(layers.modules()):
                        if isinstance(layer, nn.Linear):
                            torch.nn.init.xavier_normal_(layer.weight.data)
                            if Bias:
                                layer.bias.data.zero_()
                        if isinstance(layer, dgcnn_layer.GNN_DGCNN):
                            torch.nn.init.xavier_normal_(layer.conv_params.weight)
                            if Bias:
                                layer.conv_params.bias.data.zero_()
                        else:
                            pass
                if isinstance(layers, torch.nn.Linear):
                    torch.nn.init.xavier_normal_(layers.weight)
                    if Bias:
                        layers.bias.data.zero_()
                if isinstance(layers, (mlp_dgcnn.MLP_DGCNN)):
                    torch.nn.init.xavier_normal_(layers.conv1d_1.weight)
                    torch.nn.init.xavier_normal_(layers.conv1d_2.weight)
                    torch.nn.init.xavier_normal_(layers.linear1.weight)
                    torch.nn.init.xavier_normal_(layers.linear2.weight)

                elif isinstance(layers, (GlobalMeanPool)):
                    pass
                elif isinstance(layers, (IdenticalPool)):
                    pass

        if Weight_Initializer == 2:                                             #.      2. Kaiming Normal_.
            for i,layers in enumerate(model.children()):
                if isinstance(layers, dgcnn_gnn_layers.dgcnn_gnn_layers):
                    for j, layer in enumerate(layers.modules()):
                        if isinstance(layer, nn.Linear):
                            torch.nn.init.kaiming_normal_(layer.weight.data)
                            if Bias:
                                layer.bias.data.zero_()
                        if isinstance(layer, dgcnn_layer.GNN_DGCNN):
                            torch.nn.init.kaiming_normal_(layer.conv_params.weight)
                            if Bias:
                                layer.conv_params.bias.data.zero_()
                        else:
                            pass
                if isinstance(layers, torch.nn.Linear):
                    torch.nn.init.kaiming_normal_(layers.weight)
                    if Bias:
                        layers.bias.data.zero_()
                if isinstance(layers, (mlp_dgcnn.MLP_DGCNN)):
                    torch.nn.init.kaiming_normal_(layers.conv1d_1.weight)
                    torch.nn.init.kaiming_normal_(layers.conv1d_2.weight)
                    torch.nn.init.kaiming_normal_(layers.linear1.weight)
                    torch.nn.init.kaiming_normal_(layers.linear2.weight)

                elif isinstance(layers, (GlobalMeanPool)):
                    pass
                elif isinstance(layers, (IdenticalPool)):
                    pass

        if Weight_Initializer == 3:                                             #.      3. Uniform (0,0.1std)
            for i,layers in enumerate(model.children()):
                if isinstance(layers, dgcnn_gnn_layers.dgcnn_gnn_layers):
                    for j, layer in enumerate(layers.modules()):
                        #print("here2")
                        if isinstance(layer, nn.Linear):
                            torch.nn.init.normal_(layer.weight.data, mean, std)
                            if Bias:
                                layer.bias.data.zero_()
                        if isinstance(layer, dgcnn_layer.GNN_DGCNN):
                            torch.nn.init.normal_(layer.conv_params.weight, mean, std)
                            if Bias:
                                layer.conv_params.bias.data.zero_()
                        else:
                            pass
                if isinstance(layers, torch.nn.Linear):
                    torch.nn.init.normal_(layers.weight, mean, std)
                    if Bias:
                        layers.bias.data.zero_()
                if isinstance(layers, (mlp_dgcnn.MLP_DGCNN)):
                    torch.nn.init.normal_(layers.conv1d_1.weight, mean, std)
                    torch.nn.init.normal_(layers.conv1d_2.weight, mean, std)
                    torch.nn.init.normal_(layers.linear1.weight, mean, std)
                    torch.nn.init.normal_(layers.linear2.weight, mean, std)
                elif isinstance(layers, (GlobalMeanPool)):
                    pass
                elif isinstance(layers, (IdenticalPool)):
                    pass


    def forward(self, graph, edge_mask):

        if graph.batch is not None:
            graph_sizes = [len(graph[i].x) for i in range(len(graph))]
        else:
            graph_sizes = [len(graph.x)]


        Output_of_GNN_Layers = self.gnn_layers(graph, edge_mask)



        Output_of_GNN_Layers.retain_grad()


        sortpooled_embedings = self.sort_pool(output_of_dgcnn_layer=Output_of_GNN_Layers, batch_graphs=graph)

        sortpooled_embedings.retain_grad()


        output_conv1d_1, maxpooled_output_conv1d_1, output_conv1d_2, to_dense, ffn_1, dropout_ffn_1, ffn_2, softmaxed_ffn_2 = self.classic_conv(sortpooled_embedings=sortpooled_embedings, graph_sizes=graph_sizes)

        return Output_of_GNN_Layers, sortpooled_embedings, output_conv1d_1, maxpooled_output_conv1d_1, output_conv1d_2, to_dense, ffn_1, dropout_ffn_1, ffn_2, softmaxed_ffn_2





node_feat_size = len(dataset[0].x[0])
k=20
dgcnn_model_example = DGCNN_Model(GNN_layers=[32, 32, 32, 7], num_classes=2, mlp_act_fun='ReLu', dgcnn_act_fun='tanh',
                                  mlp_dropout_rate=0.5, Weight_Initializer=2, Bias=False, dgcnn_k=k, node_feat_size=node_feat_size,
                                  hid_channels=[16,32], conv1d_kernels=[2,5], ffn_layer_size=128, strides=[2,1])


tanh is Selected.
tanh is Selected.


In [ ]:
import sys
py_path = '/content/drive/MyDrive/Explainability Methods/Models/Script/'
sys.path.insert(0,py_path)

In [ ]:
import DGCNN as dgcnn_model
reload(dgcnn_model)

<module 'DGCNN' from '/content/drive/MyDrive/Explainability Methods/Models/Script/DGCNN.py'>

In [ ]:
dgcnn_model_example = dgcnn_model.DGCNN_Model(GNN_layers=[32, 32, 32, 7], num_classes=2, mlp_act_fun='ReLu', dgcnn_act_fun='tanh',
                                              mlp_dropout_rate=0.5, Weight_Initializer=2, Bias=True, dgcnn_k=17, node_feat_size=7,
                                              hid_channels=[16,32], conv1d_kernels=[2,5], ffn_layer_size=128, strides=[2,1])

tanh is Selected.
tanh is Selected.


In [125]:
GNN_Model_optimizer = torch.optim.Adam(dgcnn_model_example.parameters(), lr=0.01, weight_decay=1e-4)

## **Train to check**

In [120]:
from torch_geometric.loader import DataLoader
dataset = TUDataset(root='data/TUDataset', name='MUTAG')
BATCH_SIZE = 64
size_of_hidden_layers = 7

batched_dataset = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

In [121]:
criterion = torch.nn.CrossEntropyLoss()
def loss_calculations(preds, gtruth):
    loss_per_epoch = criterion(preds, gtruth)
    return loss_per_epoch


In [122]:
def train_step():
    GNN_Model_loss_batch = []
    Pred_Labels = []
    Real_Labels = []

    dgcnn_model_example.train()
    dgcnn_model_example.zero_grad()
    torch.autograd.set_detect_anomaly(True)
    for batch_of_graphs in batched_dataset:

        final_GNN_layer_output, sortpooled_embedings, output_conv1d_1, maxpooled_output_conv1d_1, output_conv1d_2, to_dense, output_h1, dropout_output_h1, output_h2, softmaxed_h2 = dgcnn_model_example(batch_of_graphs, None)
        batch_loss = loss_calculations(softmaxed_h2, batch_of_graphs.y)


        Pred_Labels.extend(softmaxed_h2.argmax(dim=1).detach().tolist())
        Real_Labels.extend(batch_of_graphs.y.detach().tolist())
        GNN_Model_loss_batch.append(batch_loss)

        batch_loss.backward()
        GNN_Model_optimizer.step()

    return torch.mean(torch.tensor(GNN_Model_loss_batch)), metrics.accuracy_score(Real_Labels, Pred_Labels)


In [123]:
#from IPython.display import Javascript  # Restrict height of output cell.
#display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))


SA_Model_training_time_per_epoch = []
SA_Model_training_Acc_per_epoch = []

def train(EPOCHS, load_index):
    SA_training_loss_per_epoch = []

    for epoch in range(EPOCHS):

        start_generation = perf_counter()
        SA_model_training_loss, training_acc = train_step()
        SA_Model_training_time_per_epoch.append(perf_counter() - start_generation)
        SA_Model_training_Acc_per_epoch.append(training_acc)

        print(f'Epoch: {epoch+1:03d}, Model Loss: {SA_model_training_loss:.4f} Accuracy: {training_acc}')

        SA_training_loss_per_epoch.append(SA_model_training_loss)


        #if (epoch + load_index + 1) % Visualization_Parameter == 0 and epoch > 0:
        #    visualize_losses(SA_training_loss_per_epoch, epoch + load_index + 1)
        #if (epoch + load_index + 1) % Model_Saving_Parameter == 0 and epoch > 0:
        #    torch.save({'epoch': epoch+load_index+1, 'model_state_dict': GNN_Model.state_dict(), 'optimizer_state_dict': GNN_Model_optimizer.state_dict(), 'loss': SA_training_loss_per_epoch,}, "/content/drive/My Drive/Explainability Methods/" + str(Explainability_name) + " on " + str(Task_name) + "/Model/" + File_Name + str(epoch + load_index + 1)+".pt")





In [126]:
EPOCHS = 3
load_index = 0
if load_index != 0:
    model_GCN, optimizer, load_index = loading_model(load_index)
train(EPOCHS, load_index)

Epoch: 001, Model Loss: 0.6556 Accuracy: 0.5691489361702128
Epoch: 002, Model Loss: 0.6334 Accuracy: 0.6648936170212766
Epoch: 003, Model Loss: 0.5162 Accuracy: 0.8085106382978723


## **check grads**

In [ ]:
def compute_grad(model, graph, with_respect):

    final_GNN_layer_output, sortpooled_embedings, output_conv1d_1, maxpooled_output_conv1d_1, output_conv1d_2, to_dense, output_h1, dropout_output_h1, output_h2, soft = model(graph, None)
    if with_respect == 1:
        loss = loss_calculations(soft, graph.y)
        #print(loss)
    elif with_respect == 2:
        loss = loss_calculations(soft, torch.tensor([0]))
        #print(loss)
    elif with_respect == 3:
        loss = loss_calculations(soft, torch.tensor([1]))
        #print(loss)

    return torch.autograd.grad(loss, list(model.parameters()), allow_unused=True)

def remove_nones(sample_grads):
    sample_grads2 = []
    for item in sample_grads:
        Each_Graph = []
        for item2 in item:
            if item2 != None:
                Each_Graph.append(torch.tensor(item2.clone().detach().requires_grad_(True), requires_grad=True))
            else:
                Each_Graph.append(torch.tensor(0))
        sample_grads2.append(Each_Graph)
    return sample_grads2

def compute_sample_grads(model, test_dataset, with_respect):

    sample_grads = [compute_grad(model, graph, with_respect) for graph in test_dataset]

    sample_grads = remove_nones(sample_grads)
    sample_grads = zip(*sample_grads)
    sample_grads = [torch.stack(shards) for shards in sample_grads]

    return sample_grads

per_sample_grads_wrt_class_one = compute_sample_grads(dgcnn_model_example, [dataset[0]], 3)
per_sample_grads_wrt_class_zero = compute_sample_grads(dgcnn_model_example, [dataset[0]], 2)
per_sample_grads_wrt_graph_label = compute_sample_grads(dgcnn_model_example, [dataset[0]], 1)

<ipython-input-15-99ef7ecbecd5>:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Each_Graph.append(torch.tensor(item2.clone().detach().requires_grad_(True), requires_grad=True))


In [ ]:
print(len(per_sample_grads_wrt_class_one))

16


In [ ]:
print(per_sample_grads_wrt_class_one)

[tensor([[[0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0.

In [ ]:
for p in dgcnn_model_example.parameters():
    print(p.grad)

tensor([[-2.3054e-12, -2.2559e-12, -2.7526e-12,  1.9104e-20, -4.3428e-32,
          2.6479e-26,  0.0000e+00],
        [ 4.1306e-12,  4.5618e-12,  5.5606e-12, -3.2547e-20,  7.9759e-32,
         -4.7281e-26,  0.0000e+00],
        [-3.4390e-12, -4.2794e-12, -5.2468e-12,  2.5296e-20, -6.4087e-32,
          4.0355e-26,  0.0000e+00],
        [-7.4334e-12, -8.8228e-12, -1.0814e-11,  6.0219e-20, -1.4083e-31,
          8.1337e-26,  0.0000e+00],
        [-3.5135e-12, -3.5920e-12, -4.3334e-12,  2.9667e-20, -6.1338e-32,
          3.8073e-26,  0.0000e+00],
        [-2.0913e-12, -2.3918e-12, -2.9663e-12,  1.6003e-20, -2.8588e-32,
          2.4334e-26,  0.0000e+00],
        [-8.6118e-12, -9.0746e-12, -1.0981e-11,  7.0795e-20, -1.4947e-31,
          9.2818e-26,  0.0000e+00],
        [-5.3819e-12, -5.8601e-12, -7.1126e-12,  4.6427e-20, -9.3636e-32,
          6.1674e-26,  0.0000e+00],
        [ 1.0522e-11,  1.0711e-11,  1.2929e-11, -8.7515e-20,  1.8070e-31,
         -1.1488e-25,  0.0000e+00],
        [ 

In [ ]:
for name, param in dgcnn_model_example.named_parameters():
    print('Name: ', name,  'Requires_Grad:' , param.requires_grad)

Name:  gnn_layers.gnn_layers.0.conv_params.weight Requires_Grad: True
Name:  gnn_layers.gnn_layers.1.conv_params.weight Requires_Grad: True
Name:  gnn_layers.gnn_layers.2.conv_params.weight Requires_Grad: True
Name:  gnn_layers.gnn_layers.3.conv_params.weight Requires_Grad: True
Name:  classic_conv.conv1d_1.weight Requires_Grad: True
Name:  classic_conv.conv1d_2.weight Requires_Grad: True
Name:  classic_conv.linear1.weight Requires_Grad: True
Name:  classic_conv.linear2.weight Requires_Grad: True


In [ ]:
print(dgcnn_model_example)

DGCNN_Model(
  (gnn_layers): dgcnn_gnn_layers(
    (gnn_layers): Sequential(
      (0): GNN_DGCNN(
        (conv_params): Linear(in_features=7, out_features=32, bias=False)
      )
      (1): GNN_DGCNN(
        (conv_params): Linear(in_features=32, out_features=32, bias=False)
      )
      (2): GNN_DGCNN(
        (conv_params): Linear(in_features=32, out_features=32, bias=False)
      )
      (3): GNN_DGCNN(
        (conv_params): Linear(in_features=32, out_features=7, bias=False)
      )
    )
  )
  (sort_pool): SortPooling_updated()
  (classic_conv): MLP_DGCNN(
    (conv1d_1): Conv1d(1, 16, kernel_size=(2,), stride=(2,), bias=False)
    (maxpool1d): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv1d_2): Conv1d(16, 32, kernel_size=(5,), stride=(1,), bias=False)
    (linear1): Linear(in_features=800, out_features=128, bias=False)
    (linear2): Linear(in_features=128, out_features=2, bias=False)
    (dropout_linear1): Dropout(p=0.5, inplace=False)
 

In [ ]:
from torch_geometric.loader import DataLoader
dataset = TUDataset(root='data/TUDataset', name='MUTAG')
batched_dataset = DataLoader(dataset, batch_size=20, shuffle=False)
print(dgcnn_model_example.classic_conv)

MLP_DGCNN(
  (conv1d_1): Conv1d(1, 16, kernel_size=(2,), stride=(2,), bias=False)
  (maxpool1d): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv1d_2): Conv1d(16, 32, kernel_size=(5,), stride=(1,), bias=False)
  (linear1): Linear(in_features=128, out_features=128, bias=False)
  (linear2): Linear(in_features=128, out_features=2, bias=False)
  (dropout_linear1): Dropout(p=0.5, inplace=False)
)


In [ ]:
for batch in batched_dataset:
    final_GNN_layer_output, sortpooled_embedings, output_conv1d_1, maxpooled_output_conv1d_1, output_conv1d_2, to_dense, ffn_1, dropout_ffn_1, ffn_2, softmaxed_ffn_2 = dgcnn_model_example(batch, None)
    print(final_GNN_layer_output.size(), sortpooled_embedings.size())

True
torch.Size([1, 17, 7]) torch.Size([20, 5, 7])
True
torch.Size([1, 17, 7]) torch.Size([20, 5, 7])
True
torch.Size([1, 17, 7]) torch.Size([20, 5, 7])
True
torch.Size([1, 17, 7]) torch.Size([20, 5, 7])
True
torch.Size([1, 17, 7]) torch.Size([20, 5, 7])
True
torch.Size([1, 17, 7]) torch.Size([20, 5, 7])
True
torch.Size([1, 17, 7]) torch.Size([20, 5, 7])
True
torch.Size([1, 17, 7]) torch.Size([20, 5, 7])
True
torch.Size([1, 17, 7]) torch.Size([20, 5, 7])
True
torch.Size([1, 17, 7]) torch.Size([8, 5, 7])


In [ ]:
for batch in dataset:
    final_GNN_layer_output, sortpooled_embedings, output_conv1d_1, maxpooled_output_conv1d_1, output_conv1d_2, to_dense, ffn_1, dropout_ffn_1, ffn_2, softmaxed_ffn_2 = dgcnn_model_example(batch, None)
    #print(final_GNN_layer_output)
    print(final_GNN_layer_output.size(), sortpooled_embedings.size())

True
torch.Size([1, 17, 7]) torch.Size([5, 7])
True
torch.Size([1, 17, 7]) torch.Size([5, 7])
True
torch.Size([1, 17, 7]) torch.Size([5, 7])
True
torch.Size([1, 17, 7]) torch.Size([5, 7])
True
torch.Size([1, 17, 7]) torch.Size([5, 7])
True
torch.Size([1, 17, 7]) torch.Size([5, 7])
True
torch.Size([1, 17, 7]) torch.Size([5, 7])
True
torch.Size([1, 17, 7]) torch.Size([5, 7])
True
torch.Size([1, 17, 7]) torch.Size([5, 7])
True
torch.Size([1, 17, 7]) torch.Size([5, 7])
True
torch.Size([1, 17, 7]) torch.Size([5, 7])
True
torch.Size([1, 17, 7]) torch.Size([5, 7])
True
torch.Size([1, 17, 7]) torch.Size([5, 7])
True
torch.Size([1, 17, 7]) torch.Size([5, 7])
True
torch.Size([1, 17, 7]) torch.Size([5, 7])
True
torch.Size([1, 17, 7]) torch.Size([5, 7])
True
torch.Size([1, 17, 7]) torch.Size([5, 7])
True
torch.Size([1, 17, 7]) torch.Size([5, 7])
True
torch.Size([1, 17, 7]) torch.Size([5, 7])
True
torch.Size([1, 17, 7]) torch.Size([5, 7])
True
torch.Size([1, 17, 7]) torch.Size([5, 7])
True
torch.Si

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
def loss_calculations(preds, gtruth):
    loss_per_epoch = criterion(preds, gtruth)
    return loss_per_epoch


In [ ]:
classifier_lr = 0.001
classifier_dropout = 0.1
classifier_weight_decay = 1e-6
GNN_Model_optimizer = torch.optim.Adam(dgcnn_model_example.parameters(), lr=classifier_lr, weight_decay=classifier_weight_decay)

In [ ]:
print(dgcnn_model_example.state_dict().keys())

odict_keys(['gnn_layers.gnn_layers.0.conv_params.weight', 'gnn_layers.gnn_layers.1.conv_params.weight', 'gnn_layers.gnn_layers.2.conv_params.weight', 'gnn_layers.gnn_layers.3.conv_params.weight', 'classic_conv.conv1d_1.weight', 'classic_conv.conv1d_2.weight', 'classic_conv.linear1.weight', 'classic_conv.linear2.weight'])


In [ ]:
#print(batched_dataset.batch_size)
dgcnn_model_example.train()
dgcnn_model_example.zero_grad()
for batch in batched_dataset:
    #print(batch)

    final_GNN_layer_output, sortpooled_embedings, output_conv1d_1, maxpooled_output_conv1d_1, output_conv1d_2, to_dense, ffn_1, dropout_ffn_1, ffn_2, softmaxed_ffn_2 = dgcnn_model_example(batch, None)

    #print(softmaxed_ffn_2)
    #break
    #print("output_conv1d_2: ", output_conv1d_2)
    #print("to_dense: ", to_dense)
    #print("ffn_1: ", ffn_1)
    #print("dropout_ffn_1: ", dropout_ffn_1)
    #print(final_GNN_layer_output)

    batch_loss = loss_calculations(softmaxed_ffn_2, batch.y)
    #print("batch_loss", batch_loss)
    #print(final_GNN_layer_output)

    batch_loss.backward()
    #nn.utils.clip_grad_norm_(dgcnn_model_example.parameters(), 1)
    #print("here: ",dgcnn_model_example.state_dict()['gnn_layers.gnn_layers.3.conv_params.weight'].grad)
    #print("classic_conv.conv1d_1.weight: ", dgcnn_model_example.state_dict()['classic_conv.linear2.weight'].grad)
    #print(dgcnn_model_example.state_dict()['dgcnn_layers.0.conv_params.weight'])
    GNN_Model_optimizer.step()

    #Pred_Labels.extend(softmaxed_ffn_2.argmax(dim=1).detach().tolist())
    #Real_Labels.extend(batch_of_graphs.y.detach().tolist())
    #GNN_Model_loss_batch.append(batch_loss)
    #print("softmax: ", softmaxed_ffn_2)


True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [ ]:
from torch_geometric.loader import DataLoader
dataset = TUDataset(root='data/TUDataset', name='MUTAG')
batched_dataset = DataLoader(dataset, batch_size=20, shuffle=False)

In [ ]:
def computational_matrices(batched_graphs, edge_mask):
    if edge_mask == None:
        tilda_adjacency_matrix = torch.tensor(to_scipy_sparse_matrix(batched_graphs.edge_index).todense()) + torch.eye(len(torch.tensor(to_scipy_sparse_matrix(batched_graphs.edge_index).todense())))
    else:
        tilda_adjacency_matrix = torch.tensor(csr_matrix((np.array(edge_mask), (np.array(batched_graphs.edge_index[0]), np.array(batched_graphs.edge_index[1])))).todense()) + torch.eye(len(torch.tensor(to_scipy_sparse_matrix(batched_graphs.edge_index).todense())))
    tilda_adjacency_matrix = tilda_adjacency_matrix.type(torch.float32)

    if batched_graphs.batch is not None:
        graph_sizes = [len(batched_graphs[i].x) for i in range(len(batched_graphs))]
        batch_size = batched_graphs.num_graphs
    else:
        graph_sizes = [len(batched_graphs.x)]
        batch_size = 1
    max_number_of_nodes_in_batch_of_graphs = max(graph_sizes)

    new_number_of_nodes = int(tilda_adjacency_matrix.size()[0] / batch_size)

    adjacency_list = []
    degree_list = []
    feature_list = []
    for i in range(batch_size):
        start = i * graph_sizes[i]
        end = (i + 1) * graph_sizes[i]
        un_padded_adj = tilda_adjacency_matrix[start:end, start:end]
        off_set = max_number_of_nodes_in_batch_of_graphs - un_padded_adj.size()[0]
        if un_padded_adj.size()[0] < max_number_of_nodes_in_batch_of_graphs:
            un_padded_adj = un_padded_adj.numpy()
            un_padded_adj = np.pad(un_padded_adj, [(0, off_set), (0, off_set)], mode='constant', constant_values=np.zeros(1,dtype=np.float32))
            un_padded_adj = torch.from_numpy(un_padded_adj)

            tilda_degree_vector = torch.sum(un_padded_adj, dim=1)
            num_nodes = tilda_degree_vector.size(0)
            tilda_degree_matrix = torch.zeros(num_nodes, num_nodes)
            tilda_degree_matrix.as_strided([num_nodes], [num_nodes + 1]).copy_(tilda_degree_vector)
        tilda_degree_matrix = tilda_degree_matrix.type(torch.float32)
        un_padded_adj = un_padded_adj.type(torch.float32)
        adjacency_list.append(un_padded_adj)
        degree_list.append(tilda_degree_matrix)

        un_padded_feat = batched_graphs.x[start:end, :].detach().numpy()
        un_padded_feat = torch.from_numpy(np.pad(un_padded_feat, [(0, off_set), (0, 0)], mode='constant', constant_values=np.zeros(1,dtype=np.float32)))
        feature_list.append(un_padded_feat)


    adjacency_list = list(map(lambda x: torch.unsqueeze(x, 0), adjacency_list))
    degree_list = list(map(lambda x: torch.unsqueeze(x, 0), degree_list))
    feature_list = list(map(lambda x: torch.unsqueeze(x, 0), feature_list))

    new_adjacecny = torch.cat(adjacency_list, dim=0)
    new_degree = torch.cat(degree_list, dim=0)
    new_features = torch.cat(feature_list, dim=0)


    return new_adjacecny, new_degree, new_features

In [ ]:
for batch in batched_dataset:
    tilda_adjacency_matrix, tilda_degree_matrix, reciprocal_tilda_degree_matrix = computational_matrices(batch, None)
    print("tilda_adjacency_matrix: ", tilda_adjacency_matrix.size())
    print("tilda_degree_matrix: ", tilda_degree_matrix.size())
    print("new_features: ", reciprocal_tilda_degree_matrix.size())
    break

tilda_adjacency_matrix:  torch.Size([20, 28, 28])
tilda_degree_matrix:  torch.Size([20, 28, 28])
new_features:  torch.Size([20, 28, 7])


In [ ]:

if batched_graphs.batch is not None:
    graph_sizes = [len(batched_graphs[i].x) for i in range(len(batched_graphs))]
    batch_size = batched_graphs.num_graphs
else:
    graph_sizes = [len(batched_graphs.x)]
    batch_size = 1
max_number_of_nodes_in_batch_of_graphs = max(graph_sizes)

new_number_of_nodes = int(joint_tilda_adjacency_matrix.size()[0] / batch_size)

adjacency_list = []
degree_list = []
feature_list = []
for i in range(batch_size):
    start = i * graph_sizes[i]
    end = (i + 1) * graph_sizes[i]
    un_padded_adj = joint_tilda_adjacency_matrix[start:end, start:end]
    off_set = max_number_of_nodes_in_batch_of_graphs - un_padded_adj.size()[0]
    if un_padded_adj.size()[0] < max_number_of_nodes_in_batch_of_graphs:
        un_padded_adj = un_padded_adj.numpy()
        un_padded_adj = np.pad(un_padded_adj, [(0, off_set), (0, off_set)], mode='constant', constant_values=np.zeros(1,dtype=np.float32))
        un_padded_adj = torch.from_numpy(un_padded_adj)

        tilda_degree_vector = torch.sum(tilda_adjacency_matrix, dim=1)
        num_nodes = tilda_degree_vector.size(0)
        tilda_degree_matrix = torch.zeros(num_nodes, num_nodes)
        tilda_degree_matrix.as_strided([num_nodes], [num_nodes + 1]).copy_(tilda_degree_vector)
    tilda_degree_matrix = tilda_degree_matrix.type(torch.float32)
    un_padded_adj = un_padded_adj.type(torch.float32)
    adjacency_list.append(un_padded_adj)
    degree_list.append(tilda_degree_matrix)

    un_padded_feat = batched_graphs.x[start:end, :].detach().numpy()
    un_padded_feat = torch.from_numpy(np.pad(un_padded_feat, [(0, off_set), (0, 0)], mode='constant', constant_values=np.zeros(1,dtype=np.float32)))
    feature_list.append(un_padded_feat)


adjacency_list = list(map(lambda x: torch.unsqueeze(x, 0), adjacency_list))
degree_list = list(map(lambda x: torch.unsqueeze(x, 0), degree_list))
feature_list = list(map(lambda x: torch.unsqueeze(x, 0), feature_list))

new_adjacecny = torch.cat(adjacency_list, dim=0)
new_degree = torch.cat(degree_list, dim=0)
new_features = torch.cat(feature_list, dim=0)


In [ ]:
print(dgcnn_model_example.state_dict()["classic_conv.conv1d_1.weight"])

None


In [ ]:
print(dgcnn_model_example)

DGCNN_Model(
  (dgcnn_layers): ModuleList(
    (0): GNN_DGCNN(
      (conv_params): Linear(in_features=7, out_features=32, bias=False)
    )
    (1-2): 2 x GNN_DGCNN(
      (conv_params): Linear(in_features=32, out_features=32, bias=False)
    )
    (3): GNN_DGCNN(
      (conv_params): Linear(in_features=32, out_features=7, bias=False)
    )
  )
  (sort_pool): SortPooling()
  (classic_conv): MLP_DGCNN(
    (conv1d_1): Conv1d(1, 16, kernel_size=(2,), stride=(2,), bias=False)
    (maxpool1d): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv1d_2): Conv1d(16, 32, kernel_size=(5,), stride=(1,), bias=False)
    (linear1): Linear(in_features=992, out_features=128, bias=False)
    (linear2): Linear(in_features=128, out_features=2, bias=False)
    (dropout_linear1): Dropout(p=0.5, inplace=False)
  )
)


In [ ]:
sum=0
for i in range(20):
    sum = sum + len(dataset[i].x)
    print(len(dataset[i].x))
print(sum)

17
13
13
19
11
28
16
20
12
17
17
20
22
13
19
22
11
17
13
18
338


In [ ]:
k = min(20, len(dataset[0].x))
dgcnn_layers = DGCNN_Model(GNN_layers=[32, 32, 32, 7], num_classes=2, mlp_act_fun='ReLu', dgcnn_act_fun='tanh', mlp_dropout_rate=0.5, Weight_Initializer=3, Bias=True, dgcnn_k=k)
print(dgcnn_layers)

tanh is Selected.
DGCNN_Model(
  (dgcnn_layers): ModuleList(
    (0): GNN_DGCNN(
      (conv_params): Linear(in_features=7, out_features=32, bias=False)
    )
    (1): GNN_DGCNN(
      (conv_params): Linear(in_features=32, out_features=32, bias=False)
    )
    (2): GNN_DGCNN(
      (conv_params): Linear(in_features=32, out_features=32, bias=False)
    )
    (3): GNN_DGCNN(
      (conv_params): Linear(in_features=32, out_features=7, bias=False)
    )
  )
  (sort_pool): SortPooling()
  (classic_conv): MLP_DGCNN(
    (conv1d_1): Conv1d(1, 16, kernel_size=(2,), stride=(2,), padding=(3,))
    (maxpool1d): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv1d_2): Conv1d(16, 32, kernel_size=(5,), stride=(1,))
    (linear1): Linear(in_features=864, out_features=128, bias=True)
    (linear2): Linear(in_features=128, out_features=2, bias=True)
    (dropout_linear1): Dropout(p=0.5, inplace=False)
  )
)


In [ ]:
print(len(dataset[0].x))
final_GNN_layer_output, sortpooled_embedings, final_output = dgcnn_layers(dataset[0])

17
None


ValueError: ignored

In [ ]:
print(final_output)

tensor([0.2232, 0.7768], grad_fn=<SoftmaxBackward0>)


In [ ]:
print(final_output)

[[-0.20715013146400452, -0.0289221853017807, -0.02776801586151123, 0.11429049074649811, -0.0, 0.25220391154289246, -0.02400323748588562, -0.23237444460391998, -0.04584074020385742, 0.023032985627651215, 0.02451241761445999, -0.144331157207489, 0.24520671367645264, 0.07532962411642075]]


In [ ]:
class GraphConvolutionLayers_DGCNN(nn.Module):
    '''
        DGCNN Model
    '''
    def __init__(self, GNN_layers, mlp_act_fun, dgcnn_act_fun, dropout_rate, Weight_Initializer, Bias, num_classes, dgcnn_k):

In [ ]:
torch.nn.Linear(bias=True, we)

In [ ]:
'''class MySpMM(torch.autograd.Function):
	@staticmethod
	def forward(ctx, sp_mat, dense_mat):
		ctx.save_for_backward(sp_mat, dense_mat)
		return torch.mm(sp_mat, dense_mat)

	@staticmethod
	def backward(ctx, grad_output):
		sp_mat, dense_mat = ctx.saved_variables
		grad_matrix1 = grad_matrix2 = None

		assert not ctx.needs_input_grad[0]
		if ctx.needs_input_grad[1]:
			grad_matrix2 = Variable(torch.mm(sp_mat.data.t(), grad_output.data))

		return grad_matrix1, grad_matrix2


def gnn_spmm(sp_mat, dense_mat):
	return MySpMM.apply(sp_mat, dense_mat)'''

In [ ]:
'''
import os
import sys
import numpy as np
import torch
import torch.nn as nn

class SortPooling_updated(nn.Module):
    def __init__(self, sort_pooling_k, node_feat_size):
        super(SortPooling_updated, self).__init__()
        self.sort_pooling_k, = sort_pooling_k,
        self.node_feat_size = node_feat_size


    def max_sort(self, row):
        return max(row)


    def forward(self, output_of_dgcnn_layer, batch_graphs):
        if batch_graphs.batch is not None:
            graph_sizes = [len(batch_graphs[i].x) for i in range(len(batch_graphs))]
        else:
            graph_sizes = [len(batch_graphs.x)]
        batch_sortpooling_zero_graphs = torch.zeros(len(graph_sizes), self.sort_pooling_k, self.node_feat_size)
        #print("batch of graphs reshaped: ", batch_sortpooling_zero_graphs.size())

        start_index = 0
        batch_graphs_ready_to_sort = []
        if batch_graphs.batch is not None:
            for i in range(batch_graphs.num_graphs):
                #graph_to_sort = output_of_dgcnn_layer[start_index: start_index + graph_sizes[i]]
                #graph_to_sort = torch.reshape(graph_to_sort, (graph_sizes[i], self.node_feat_size))

                sorted_graph = torch.Tensor(sorted(output_of_dgcnn_layer[i].detach().numpy(), key = lambda row : max(row), reverse=True))
                if sorted_graph.size()[0] >= self.sort_pooling_k:
                    batch_sortpooling_zero_graphs[i][:self.sort_pooling_k, 0:self.node_feat_size] = sorted_graph[:self.sort_pooling_k, 0:self.node_feat_size]
                else:
                    batch_sortpooling_zero_graphs[i][:sorted_graph.size()[0], 0:self.node_feat_size] = sorted_graph[:sorted_graph.size()[0], 0:self.node_feat_size]
                sorted_graph.requires_grad = True
                batch_graphs_ready_to_sort.append(sorted_graph)
                #strat_index = start_index + graph_sizes[i]
        else:
            for i in range(len([batch_graphs])):
                #graph_to_sort = output_of_dgcnn_layer[start_index: start_index + graph_sizes[i]]
                #graph_to_sort = torch.reshape(graph_to_sort, (graph_sizes[i], self.node_feat_size))

                sorted_graph = torch.Tensor(sorted(output_of_dgcnn_layer[i].cpu().detach().tolist(), key = lambda row : max(row), reverse=True))
                if sorted_graph.size()[0] >= self.sort_pooling_k:
                    batch_sortpooling_zero_graphs[i][:self.sort_pooling_k, 0:self.node_feat_size] = sorted_graph[:self.sort_pooling_k, 0:self.node_feat_size]
                else:
                    batch_sortpooling_zero_graphs[i][:sorted_graph.size()[0], 0:self.node_feat_size] = sorted_graph[:sorted_graph.size()[0], 0:self.node_feat_size]

                batch_graphs_ready_to_sort.append(sorted_graph)
                #strat_index = start_index + graph_sizes[i]

        return batch_sortpooling_zero_graphs
'''

In [ ]:
'''import torch
import torch.nn as nn
from torch.autograd import Variable
import math
import torch.nn.functional as F
from torch.nn.parameter import Parameter
from torch_geometric.utils.convert import to_scipy_sparse_matrix
from torch_geometric.utils.train_test_split_edges import torch_geometric
import networkx as nx
import numpy as np
from torch_geometric.nn import GCNConv
import sys
from torch_geometric.datasets import TUDataset
py_path = '/content/drive/MyDrive/Explainability Methods/Models/Script/Layers/'
sys.path.insert(0,py_path)
#import matrix_util as Mat_Util
#import DGCNN_layer as dgcnn_layer
#import DGCNN_GNN_Layers as dgcnn_gnn_layers
#import SortPooling_Layer as sortpooling_layer
#import MLP_DGCNN as mlp_dgcnn

class GlobalMeanPool(nn.Module):

    def __init__(self):
        super().__init__()

    def forward(self, x, batch):
        return gnn.global_mean_pool(x, batch)
################################################################################
class IdenticalPool(nn.Module):

    def __init__(self):
        super().__init__()

    def forward(self, x, batch):
        return x

################################################################################
class DGCNN_Model(nn.Module):
    '''
        DGCNN Layers using sparse adjacency matrix
    '''
    def __init__(self, GNN_layers, mlp_act_fun, dgcnn_act_fun, mlp_dropout_rate, Weight_Initializer, Bias, num_classes, dgcnn_k,
                 node_feat_size, hid_channels, conv1d_kernels, ffn_layer_size, strides):

        super(DGCNN_Model, self).__init__()
        self.GNN_layers = GNN_layers
        self.output_dim = GNN_layers[-1]
        self.num_GNN_layers = len(GNN_layers)
        self.mlp_dropout_rate = mlp_dropout_rate
        self.Bias = Bias
        self.Weight_Initializer = Weight_Initializer
        self.dgcnn_k = dgcnn_k
        self.node_feat_size = node_feat_size
        self.num_classes = num_classes
        self.hid_channels = hid_channels
        self.conv1d_kernels = conv1d_kernels
        self.ffn_layer_size = ffn_layer_size
        self.strides = strides

        self.gnn_layers = dgcnn_gnn_layers(GNN_layers=[32, 32, 32, 7], node_feat_size=self.node_feat_size,
                                                            Bias=self.Bias, dgcnn_act_fun=dgcnn_act_fun)

        self.sort_pool = SortPooling_updated(self.dgcnn_k, node_feat_size=self.node_feat_size)

        self.classic_conv = MLP_DGCNN(num_class=self.num_classes, node_feat_size=self.node_feat_size,
                                                mlp_act_fun=mlp_act_fun, dropout_rate=self.mlp_dropout_rate,
                                                hid_channels=self.hid_channels, conv1d_kernels=self.conv1d_kernels,
                                                dgcnn_k=self.dgcnn_k, ffn_layer_size=self.ffn_layer_size, Bias=self.Bias,
                                                strides=self.strides)
        if dgcnn_act_fun == 'ReLu':
            self.dgcnn_act_fun = F.relu
            print('ReLu is Selected.')
        elif dgcnn_act_fun == 'eLu':
            self.dgcnn_act_fun = nn.functional.elu
            print('eLu is Selected.')
        elif dgcnn_act_fun == 'tanh':
            self.dgcnn_act_fun = torch.tanh
            print('tanh is Selected.')



        mean = 0
        std = 0.1
        #self.initialize_weights(Weight_Initializer, Bias, mean, std)


    def initialize_weights(model, Weight_Initializer, Bias, mean, std):
        # 1. Xavier Normal_.  2. Kaiming Normal_.  3. Uniform (0,0.1std)
        if Weight_Initializer == 1:                                             #.      1. Xavier Normal_.
            for i,layers in enumerate(model.children()):
                if isinstance(layers, dgcnn_gnn_layers.dgcnn_gnn_layers):
                    for j, layer in enumerate(layers.modules()):
                        if isinstance(layer, nn.Linear):
                            torch.nn.init.xavier_normal_(layer.weight.data)
                            if Bias:
                                layer.bias.data.zero_()
                        if isinstance(layer, dgcnn_layer.GNN_DGCNN):
                            torch.nn.init.xavier_normal_(layer.conv_params.weight)
                            if Bias:
                                layer.conv_params.bias.data.zero_()
                        else:
                            pass
                if isinstance(layers, torch.nn.Linear):
                    torch.nn.init.xavier_normal_(layers.weight)
                    if Bias:
                        layers.bias.data.zero_()
                if isinstance(layers, (mlp_dgcnn.MLP_DGCNN)):
                    torch.nn.init.xavier_normal_(layers.conv1d_1.weight)
                    torch.nn.init.xavier_normal_(layers.conv1d_2.weight)
                    torch.nn.init.xavier_normal_(layers.linear1.weight)
                    torch.nn.init.xavier_normal_(layers.linear2.weight)

                elif isinstance(layers, (GlobalMeanPool)):
                    pass
                elif isinstance(layers, (IdenticalPool)):
                    pass

        if Weight_Initializer == 2:                                             #.      2. Kaiming Normal_.
            for i,layers in enumerate(model.children()):
                if isinstance(layers, dgcnn_gnn_layers.dgcnn_gnn_layers):
                    for j, layer in enumerate(layers.modules()):
                        if isinstance(layer, nn.Linear):
                            torch.nn.init.kaiming_normal_(layer.weight.data)
                            if Bias:
                                layer.bias.data.zero_()
                        if isinstance(layer, dgcnn_layer.GNN_DGCNN):
                            torch.nn.init.kaiming_normal_(layer.conv_params.weight)
                            if Bias:
                                layer.conv_params.bias.data.zero_()
                        else:
                            pass
                if isinstance(layers, torch.nn.Linear):
                    torch.nn.init.kaiming_normal_(layers.weight)
                    if Bias:
                        layers.bias.data.zero_()
                if isinstance(layers, (mlp_dgcnn.MLP_DGCNN)):
                    torch.nn.init.kaiming_normal_(layers.conv1d_1.weight)
                    torch.nn.init.kaiming_normal_(layers.conv1d_2.weight)
                    torch.nn.init.kaiming_normal_(layers.linear1.weight)
                    torch.nn.init.kaiming_normal_(layers.linear2.weight)

                elif isinstance(layers, (GlobalMeanPool)):
                    pass
                elif isinstance(layers, (IdenticalPool)):
                    pass

        if Weight_Initializer == 3:                                             #.      3. Uniform (0,0.1std)
            for i,layers in enumerate(model.children()):
                if isinstance(layers, dgcnn_gnn_layers.dgcnn_gnn_layers):
                    for j, layer in enumerate(layers.modules()):
                        #print("here2")
                        if isinstance(layer, nn.Linear):
                            torch.nn.init.normal_(layer.weight.data, mean, std)
                            if Bias:
                                layer.bias.data.zero_()
                        if isinstance(layer, dgcnn_layer.GNN_DGCNN):
                            torch.nn.init.normal_(layer.conv_params.weight, mean, std)
                            if Bias:
                                layer.conv_params.bias.data.zero_()
                        else:
                            pass
                if isinstance(layers, torch.nn.Linear):
                    torch.nn.init.normal_(layers.weight, mean, std)
                    if Bias:
                        layers.bias.data.zero_()
                if isinstance(layers, (mlp_dgcnn.MLP_DGCNN)):
                    torch.nn.init.normal_(layers.conv1d_1.weight, mean, std)
                    torch.nn.init.normal_(layers.conv1d_2.weight, mean, std)
                    torch.nn.init.normal_(layers.linear1.weight, mean, std)
                    torch.nn.init.normal_(layers.linear2.weight, mean, std)
                elif isinstance(layers, (GlobalMeanPool)):
                    pass
                elif isinstance(layers, (IdenticalPool)):
                    pass


    def forward(self, graph, edge_mask):
        #x, edge_index, batch, y = graph.x, graph.edge_index, graph.batch, graph.y

        if graph.batch is not None:
            graph_sizes = [len(graph[i].x) for i in range(len(graph))]
        else:
            graph_sizes = [len(graph.x)]


        #x = x.type(torch.float32)

        Output_of_GNN_Layers = self.gnn_layers(graph, edge_mask)
        #final_GNN_layer_output = Output_of_GNN_Layers[-1]


        Output_of_GNN_Layers.retain_grad()
        #print(len(Output_of_GNN_Layers[0]))
        #print("final_GNN_layer_output: ",final_GNN_layer_output.size())

        sortpooled_embedings = self.sort_pool(output_of_dgcnn_layer=Output_of_GNN_Layers, batch_graphs=graph)
        #print("sortpooling: ",sortpooled_embedings.size())
        #print("sortpooled_embedings: ",sortpooled_embedings)
        sortpooled_embedings.retain_grad()


        output_conv1d_1, maxpooled_output_conv1d_1, output_conv1d_2, to_dense, ffn_1, dropout_ffn_1, ffn_2, softmaxed_ffn_2 = self.classic_conv(sortpooled_embedings=sortpooled_embedings, graph_sizes=graph_sizes)

        return Output_of_GNN_Layers, sortpooled_embedings, output_conv1d_1, maxpooled_output_conv1d_1, output_conv1d_2, to_dense, ffn_1, dropout_ffn_1, ffn_2, softmaxed_ffn_2





node_feat_size = len(dataset[0].x[0])
k=20
dgcnn_model_example = DGCNN_Model(GNN_layers=[32, 32, 32, 7], num_classes=2, mlp_act_fun='ReLu', dgcnn_act_fun='tanh',
                                  mlp_dropout_rate=0.5, Weight_Initializer=2, Bias=False, dgcnn_k=k, node_feat_size=node_feat_size,
                                  hid_channels=[16,32], conv1d_kernels=[2,5], ffn_layer_size=128, strides=[2,1])

#print(dgcnn_model_example)
#final_GNN_layer_output, sortpooled_embedings, output_conv1d_1, maxpooled_output_conv1d_1, output_conv1d_2, to_dense, output_h1, dropout_output_h1, output_h2, softmaxed_h2 = dgcnn_model_example(dataset[0])
#print(softmaxed_h2)'''